In [1]:
import json

sampled_train_path = "/data/aofei/hallucination/MIMIC_CXR/sampled_paths_train.json"
sampled_test_path = "/data/aofei/hallucination/MIMIC_CXR/sampled_paths_test.json"

with open(sampled_train_path, "r") as f:
    sampled_train_paths = json.load(f) 

with open(sampled_test_path, "r") as f: 
    sampled_test_paths = json.load(f)

In [2]:
len(sampled_train_paths), len(sampled_test_paths)

(2000, 2000)

In [3]:
set(sampled_train_paths) & set(sampled_test_paths)

set()

In [4]:
import os
mimic_train = []
s = 0
report_prompt_MIMIC = "Analyze the chest X-ray and generate a concise medical report."
all_reports = []
for p in sampled_train_paths:
    template = {}
    template["id"] = s
    template["image"] = p
    s += 1
    report_path = "/".join(p.split("/")[:3]) + ".txt"
    report_path = os.path.join("/data/aofei/hallucination/MIMIC_CXR/reports_files", report_path)
    with open(report_path, "r") as f:
        report = f.read()
    # report = report.split("FINDINGS:")[-1]
    # report = "FINDINGS:" + report
    all_reports.append(report)
    template['conversations'] = []
    new_qa = {"from": "human", "value": "<image>\n" + report_prompt_MIMIC}
    new_qa2 = {"from": "gpt", "value": report}
    template['conversations'] += [new_qa, new_qa2]
    mimic_train.append(template)

In [5]:
all_reports[:2]

['                                 FINAL REPORT\n EXAMINATION:  CHEST (PORTABLE AP)\n \n INDICATION:  ___ yo M struck by vehicle, R SDH s/p evac crani, L epidural\n hematoma untreated s/p tib fx ex fix  // fever. r/o PNA\n \n COMPARISON:  ___\n \n IMPRESSION: \n \n As compared to the previous radiograph, no relevant change has occurred. The\n lung parenchyma is clear and the current image shows no evidence of pneumonia.\n A minimal atelectasis at the right hilus is less severe than on the previous\n exam.  No larger pleural effusions.  No pneumothorax.  Normal size of the\n cardiac silhouette.\n',
 ' WET READ: ___ ___ ___ 9:31 AM\n  \n  \n  \n  Left lung base consolidation and small left pleural effusion appears similar\n  to CT from ___.\n WET READ VERSION #1 ___ ___ ___ 1:36 AM\n  Left lung base consolidation and small left pleural effusion appears similar\n  to CT from ___.\n ______________________________________________________________________________\n                            

In [6]:
import json
import csv
train_path = '/data/aofei/hallucination/MIMIC_CXR/data_report/training_no_clean.json'
test_path = '/data/aofei/hallucination/MIMIC_CXR/data_report/test.json'
# Save training data
print(len(mimic_train))
with open(train_path, 'w') as train_file:
    json.dump(mimic_train, train_file, indent=4)

# # Save test data
# print(len(new_iu_test_data))
# with open(test_path, 'w') as test_file:
#     json.dump(new_iu_test_data, test_file, indent=4)

2000


## Process the report

In [9]:
import section_parser as sp
def list_rindex(l, s):
    """Helper function: *last* matching element in a list"""
    return len(l) - l[-1::-1].index(s) - 1

In [8]:
sp.section_text(all_reports[0])

(['                                 FINAL REPORT\n ',
  ' CHEST (PORTABLE AP)\n ',
  ' ___ yo M struck by vehicle, R SDH s/p evac crani, L epidural\n hematoma untreated s/p tib fx ex fix  // fever. r/o PNA\n ',
  ' ___\n ',
  '\n \n As compared to the previous radiograph, no relevant change has occurred. The\n lung parenchyma is clear and the current image shows no evidence of pneumonia.\n A minimal atelectasis at the right hilus is less severe than on the previous\n exam.  No larger pleural effusions.  No pneumothorax.  Normal size of the\n cardiac silhouette.\n'],
 ['preamble', 'examination', 'indication', 'comparison', 'impression'],
 [0, 60, 96, 230, 250])

In [91]:
cleaned_reports = []
batch_data_dict = {}

In [113]:
cleaned_reports_test = []
batch_data_dict_test = {}

In [13]:
import os
all_cleaned_reports = []
# for i in range(0, len(all_reports)):
for i in range(0, len(all_reports)):
    report = all_reports[i]
    sections, section_names, section_idx = sp.section_text(report)
    idx = -1
    cleaned_report = ""
    if "impression" not in section_names and "findings" not in section_names:
        all_cleaned_reports.append("")
    else:
        if "impression" in section_names:
            idx = section_names.index("impression")
            cleaned_report += sections[idx]
        if "findings" in section_names:
            idx = section_names.index("findings")
            cleaned_report += sections[idx]
        all_cleaned_reports.append(cleaned_report)

In [14]:
all_cleaned_reports[:3]

['\n \n As compared to the previous radiograph, no relevant change has occurred. The\n lung parenchyma is clear and the current image shows no evidence of pneumonia.\n A minimal atelectasis at the right hilus is less severe than on the previous\n exam.  No larger pleural effusions.  No pneumothorax.  Normal size of the\n cardiac silhouette.\n',
 '\n \n In comparison with the study of ___, the tip of the PICC line now appears\n to be at the level of the cavoatrial junction.  Pulmonary vascularity is\n essentially within normal limits.  Opacification at the left base is again\n seen, representing pleural fluid and either atelectasis or superimposed\n pneumonia given the clinical history.\n',
 '\n \n In comparison with the earlier study of this date, there has been placement of\n a nasogastric tube that extends at least to the mid to upper body of the\n stomach, were crosses the lower margin of the image.  Otherwise, little\n change.\n']

In [17]:
no_rep_ids = []
for i in range(len(all_cleaned_reports)):
    rep = all_cleaned_reports[i]
    if rep == "" or len(rep) < 10:
        no_rep_ids.append(i)

In [41]:
def clean_report_func(text):
    text = text.replace('\t', '')
    text = text.replace('\n', '')

    """delete note like 'talk to Doc, at pm, where'"""
    sentence_list = text.split('.')
    # "with Dr", "by Dr", "to Dr",
    key_words = ["email", "phone", "Dr", "contact", "discuss", "minutes", "review", "dictation", "observation",
                    "communi"]

    first_cut_pos = 0
    temp_key = []
    for sentence_index, single_sentence in enumerate(sentence_list):
        for keyy in key_words:
            if keyy in single_sentence:
                temp_key.append(keyy)
                # find_pos = single_sentence.find(keyy)
                # first_cut_pos += find_pos
                break
            # else:
            #     if len(temp_key) != 0:
            #         first_cut_pos += len(single_sentence)
        if len(temp_key) != 0:
            break

    if len(temp_key) == 0:
        new_imp_text = text
    else:
        for ii in range(sentence_index):
            first_cut_pos += (len(sentence_list[ii]) + 1)
        new_imp_text = text[:first_cut_pos]
    
    return new_imp_text.strip()


In [30]:
all_cleaned_reports2 = [clean_report_func(rep) for rep in all_cleaned_reports]

In [31]:
no_rep_ids = []
for i in range(len(all_cleaned_reports)):
    rep = all_cleaned_reports[i]
    if rep == "" or len(rep) < 10:
        no_rep_ids.append(i)
len(no_rep_ids)

82

In [32]:
all_cleaned_reports2[:3]

['As compared to the previous radiograph, no relevant change has occurred. The lung parenchyma is clear and the current image shows no evidence of pneumonia. A minimal atelectasis at the right hilus is less severe than on the previous exam.  No larger pleural effusions.  No pneumothorax.  Normal size of the cardiac silhouette.',
 'In comparison with the study of ___, the tip of the PICC line now appears to be at the level of the cavoatrial junction.  Pulmonary vascularity is essentially within normal limits.  Opacification at the left base is again seen, representing pleural fluid and either atelectasis or superimposed pneumonia given the clinical history.',
 'In comparison with the earlier study of this date, there has been placement of a nasogastric tube that extends at least to the mid to upper body of the stomach, were crosses the lower margin of the image.  Otherwise, little change.']

In [34]:
mimic_train_cleaned = []
for i in mimic_train:
    k = i.copy()
    clean_report = all_cleaned_reports2[i['id']]
    if clean_report == "" or len(clean_report) < 10:
        continue
    k['conversations'][-1]['value'] = clean_report
    mimic_train_cleaned.append(k)

In [35]:
len(mimic_train_cleaned), mimic_train_cleaned[-1]

(1911,
 {'id': 1998,
  'image': 'p13/p13202007/s55485353/19e461d1-03e672cd-f3ec31aa-89d44e06-7f6f6f05.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
   {'from': 'gpt',
    'value': '1. Appropriate positioning of the ETT and NG tube. 2. Multifocal pneumonia with mild improvement in the right upper and lower lobes.  ET tube terminates 4 cm above the carina.  There is a NG tube with the tip and side hole in the stomach.  There is bilateral diffuse opacification, consistent with multifocal pneumonia seen on the prior chest CT.  The opacification appears to improved in the right upper and lower lobes.  Heart size is stable.  The mediastinal and hilar contours are stable. The pulmonary vasculature is normal.  No pleural effusion or pneumothorax is seen. There are no acute osseous abnormalities.'}]})

In [36]:
import json
import csv
train_path = '/data/aofei/hallucination/MIMIC_CXR/data_report/training.json'
# test_path = '/data/aofei/hallucination/MIMIC_CXR/data_report/test.json'
# Save training data
print(len(mimic_train_cleaned))
with open(train_path, 'w') as train_file:
    json.dump(mimic_train_cleaned, train_file, indent=4)


1911


In [37]:
import os
mimic_test = []
s = 0
report_prompt_MIMIC = "Analyze the chest X-ray and generate a concise medical report."
all_test_reports = []
for p in sampled_test_paths[:500]:
    template = {}
    template["id"] = s
    template["image"] = p
    s += 1
    report_path = "/".join(p.split("/")[:3]) + ".txt"
    report_path = os.path.join("/data/aofei/hallucination/MIMIC_CXR/reports_files", report_path)
    with open(report_path, "r") as f:
        report = f.read()
    # report = report.split("FINDINGS:")[-1]
    # report = "FINDINGS:" + report
    image_exist_flag = os.path.exists(os.path.join("/data/aofei/hallucination/mimic_cxr/images", p))
    if not image_exist_flag:
        print(p)
    all_test_reports.append(report)
    template['conversations'] = []
    new_qa = {"from": "human", "value": "<image>\n" + report_prompt_MIMIC}
    new_qa2 = {"from": "gpt", "value": report}
    template['conversations'] += [new_qa, new_qa2]
    mimic_test.append(template)

In [38]:
len(mimic_test), mimic_test[0]

(500,
 {'id': 0,
  'image': 'p19/p19454978/s57331547/7d047120-d24a497e-fc26ea7e-6c3acc0c-ce5bc190.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
   {'from': 'gpt',
    'value': "                                 FINAL REPORT\n EXAMINATION:  CHEST (PORTABLE AP)\n \n INDICATION:  ___ year old woman with Caroli's disease, here with sepsis, new\n fever spike, ? new lung pathology  // ? pneumonia\n \n TECHNIQUE:  Portable chest film\n \n COMPARISON:   ___\n \n FINDINGS: \n \n There is a new retrocardiac opacity. A right IJ has been removed. Small\n bilateral pleural effusions are seen. Cardiomediastinal silhouette is\n unchanged compared to prior.\n \n IMPRESSION: \n \n New retrocardiac opacity concerning for pneumonia in the appropriate clinical\n setting.\n \n NOTIFICATION:   The findings were relayed by text page by Dr. ___ with Dr.\n ___ on the telephone on ___ at 12:15 PM, 1 minutes after discovery of\

In [39]:
import os
all_cleaned_test_reports = []
# for i in range(0, len(all_reports)):
for i in range(0, len(all_test_reports)):
    report = all_test_reports[i]
    sections, section_names, section_idx = sp.section_text(report)
    idx = -1
    cleaned_report = ""
    if "impression" not in section_names and "findings" not in section_names:
        all_cleaned_test_reports.append("")
    else:
        if "impression" in section_names:
            idx = section_names.index("impression")
            cleaned_report += sections[idx]
        if "findings" in section_names:
            idx = section_names.index("findings")
            cleaned_report += sections[idx]
        all_cleaned_test_reports.append(cleaned_report)

In [42]:
all_cleaned_test_reports2 = [clean_report_func(rep) for rep in all_cleaned_test_reports]

no_rep_ids = []
for i in range(len(all_cleaned_test_reports)):
    rep = all_cleaned_test_reports[i]
    if rep == "" or len(rep) < 10:
        no_rep_ids.append(i)
len(no_rep_ids)

56

In [43]:
mimic_test_cleaned = []
for i in mimic_test:
    k = i.copy()
    clean_report = all_cleaned_test_reports2[i['id']]
    if clean_report == "" or len(clean_report) < 10:
        continue
    k['conversations'][-1]['value'] = clean_report
    mimic_test_cleaned.append(k)

In [45]:
len(mimic_test_cleaned), mimic_test_cleaned[-1]

(441,
 {'id': 499,
  'image': 'p16/p16043637/s52793175/1b3d4f71-68977c5e-a070ff6b-29584c84-b70bf667.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
   {'from': 'gpt',
    'value': "No acute cardiopulmonary process, unchanged compared to ___. PA and lateral views of the chest.  A left-sided pacemaker is in appropriate position.  Sternotomy wires again seen.  An aortic valve replacement is again noted.  Faint haziness over the lower lung fields bilaterally, likely from patient's body habitus.  This is unchanged.  There is no new focal consolidation, pleural effusion or pneumothorax.  Cardiomediastinal and hilar contours are normal."}]})

In [121]:
mimic_test_cleaned[-1]

{'id': 499,
 'image': 'p16/p16043637/s52793175/1b3d4f71-68977c5e-a070ff6b-29584c84-b70bf667.jpg',
 'conversations': [{'from': 'human',
   'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
  {'from': 'gpt',
   'value': 'PA and lateral views show a left-sided pacemaker, sternotomy wires, and an aortic valve replacement in place. There is faint haziness over the lower lung fields bilaterally, likely due to body habitus, with no new focal consolidation, pleural effusion, or pneumothorax. Cardiomediastinal and hilar contours are normal. No acute cardiopulmonary process detected.'}]}

In [46]:
import json
import csv
test_path = '/data/aofei/hallucination/MIMIC_CXR/data_report/test.json'
# Save training data
print(len(mimic_test_cleaned))
with open(test_path, 'w') as train_file:
    json.dump(mimic_test_cleaned, train_file, indent=4)


441


In [47]:
mimic_test_cleaned[100], mimic_test[100]

({'id': 115,
  'image': 'p12/p12185775/s56494283/957c26f1-18da168e-71c98f71-7f791b2a-4cb759cb.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
   {'from': 'gpt',
    'value': 'Severe cardiomegaly and widened mediastinum are unchanged. Pulmonary edema has markedly improved. Retrocardiac opacities have improved consistent with improving atelectasis and small left effusion. There is no pneumothorax. Lines and tubes are in standard position. calcified granulomas in the left upper lobe are again noted.'}]},
 {'id': 100,
  'image': 'p15/p15857729/s52428322/754c8b94-ddf3a484-279e5c47-973dad5c-3e52b57c.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
   {'from': 'gpt', 'value': 'No acute cardiopulmonary process. ___.'}]})

In [48]:
## Porcess training and test data

import json

train_path = '/data/aofei/hallucination/MIMIC_CXR/data_report/training.json'
train_mask_path = '/data/aofei/hallucination/MIMIC_CXR/data_report/training_masks_top4.json'

train_data = json.load(open(train_path, 'r'))
train_mask_data = json.load(open(train_mask_path, 'r'))

In [53]:
len(train_mask_data)
new_train_data = []
mask_dict = dict(zip([i['image'] for i in train_mask_data], [i['bboxes'] for i in train_mask_data]))
for i in train_data:
    if i['image'] not in mask_dict:
        continue
    i['bboxes'] = mask_dict[i['image']]
    i['masks'] = []
    new_train_data.append(i)

In [54]:
len(new_train_data), new_train_data[-1]

(1902,
 {'id': 1998,
  'image': 'p13/p13202007/s55485353/19e461d1-03e672cd-f3ec31aa-89d44e06-7f6f6f05.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
   {'from': 'gpt',
    'value': '1. Appropriate positioning of the ETT and NG tube. 2. Multifocal pneumonia with mild improvement in the right upper and lower lobes.  ET tube terminates 4 cm above the carina.  There is a NG tube with the tip and side hole in the stomach.  There is bilateral diffuse opacification, consistent with multifocal pneumonia seen on the prior chest CT.  The opacification appears to improved in the right upper and lower lobes.  Heart size is stable.  The mediastinal and hilar contours are stable. The pulmonary vasculature is normal.  No pleural effusion or pneumothorax is seen. There are no acute osseous abnormalities.'}],
  'bboxes': [[74, 25, 49, 62],
   [160, 27, 41, 65],
   [165, 27, 37, 56],
   [165, 27, 40, 103]],
  'masks': 

In [55]:
# with open(train_path, 'w') as train_file:
#     json.dump(new_train_data, train_file, indent=4)
with open(train_mask_path, 'w') as train_file:
    json.dump(new_train_data, train_file, indent=4)

In [12]:
test_path = '/data/aofei/hallucination/MIMIC_CXR/data_report/test.json'
test_data = json.load(open(test_path, 'r'))
len(test_data), test_data[0], test_data[-1]

(500,
 {'id': 0,
  'image': 'p19/p19454978/s57331547/7d047120-d24a497e-fc26ea7e-6c3acc0c-ce5bc190.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
   {'from': 'gpt',
    'value': 'There is a new retrocardiac opacity, and small bilateral pleural effusions are seen. New retrocardiac opacity is concerning for pneumonia in the appropriate clinical setting.'}]},
 {'id': 499,
  'image': 'p16/p16043637/s52793175/1b3d4f71-68977c5e-a070ff6b-29584c84-b70bf667.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
   {'from': 'gpt',
    'value': 'PA and lateral views show a left-sided pacemaker, sternotomy wires, and an aortic valve replacement in place. There is faint haziness over the lower lung fields bilaterally, likely due to body habitus, with no new focal consolidation, pleural effusion, or pneumothorax. Cardiomediastinal and hila

In [13]:
new_test_data = []
_id = 0
for i in range(len(test_data)):
    ground_truth = test_data[i]['conversations'][-1]['value']
    if len(ground_truth) < 10:
        print(i, len(ground_truth))
        continue
    test_data[i]["id"] = _id
    _id += 1
    new_test_data.append(test_data[i])

165 0
440 0


In [14]:
new_test_data[0], len(new_test_data), new_test_data[-1]

({'id': 0,
  'image': 'p19/p19454978/s57331547/7d047120-d24a497e-fc26ea7e-6c3acc0c-ce5bc190.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
   {'from': 'gpt',
    'value': 'There is a new retrocardiac opacity, and small bilateral pleural effusions are seen. New retrocardiac opacity is concerning for pneumonia in the appropriate clinical setting.'}]},
 498,
 {'id': 497,
  'image': 'p16/p16043637/s52793175/1b3d4f71-68977c5e-a070ff6b-29584c84-b70bf667.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nAnalyze the chest X-ray and generate a concise medical report.'},
   {'from': 'gpt',
    'value': 'PA and lateral views show a left-sided pacemaker, sternotomy wires, and an aortic valve replacement in place. There is faint haziness over the lower lung fields bilaterally, likely due to body habitus, with no new focal consolidation, pleural effusion, or pneumothorax. Cardiomediastinal and hila

In [15]:
with open(test_path, 'w') as test_file:
    json.dump(new_test_data, test_file, indent=4)